Decorators
==========

Decorators are a built-in feature that allow you to add predefined functionality on top of other functions.  So, what exactly is a decorator?  The general definitions is that a decorator is a callable object that takes a callable object as an argument and returns a callable object.

Whew!  That was a mouthful.  Since most of the callable objects we'll be working with are functions, let's simplify that definition.  A decorator is a function which takes a function as an argument and returns a function.

Why would you ever want to use something like that?  Let's consider a simple but useful example.

In [ ]:
import time

def timer(func):
    def new_func(*args, **kwargs):
        start = time.time()
        value = func(*args, **kwargs)
        duration = time.time() - start
        print(f'Running {func.__name__} took {duration} seconds.')
        return value
    return new_func

Before we study `timer` and figure out what it does, let's recognize that it fits the definition of a decorator.  It is a function, it takes in a function (`func`) as an argument, and it returns a function (`new_func`).

Okay, so what does `new_func` do?  It makes a note of the current time, calls `func`, figures out how many seconds elapsed while `func` was running, prints that result to the screen, and returns `func`'s value.

By the way, if you're confused by the `*args` and `**kwargs`, they represent the positional and keyword arguments, respectively, that `new_func` is called with.  The idea is that we don't know what arguments the user will pass, so we're telling Python, "However many positional and keyword arguments someone calls `new_func` with, pass those same arguments to `func`."

Okay, so let's see `timer` in action.

In [ ]:
def slow_add(x, y):
    result = x + y
    time.sleep(result)
    return result

timed_slow_add = timer(slow_add)
timed_slow_add(2, 3)

Pretty neat, huh?  We now have a reusable piece of code (`timer`) that we can apply to any function.  The returned function wraps the original function in that it performs the exact same functionality plus we get to know how long the call took.

However, there's some klunkiness we need to deal with.  Specifically, the name `timed_slow_add` is too cumbersome.  In addition, there's no reason for us to carry around two separate variables, `slow_add` and `timed_slow_add`, when we'll only ever be calling `timed_slow_add`.

What if we did
```python
slow_add = timer(slow_add)
```
?

Not only does that avoid cluttering our global namespace with a variable we're never going to use but it allows us to easily modify our code in case we ever want to remove the added functionality.

To explain, consider what we'd have to do if we used the name `timed_slow_add`.  All throughout our code, we'd have to type out `timed_slow_add` every time we wanted to use our original function.  Remember the lesson that Larry Wall taught us: The first great virtue of a programmer is laziness.

Furthermore, what if we suddenly wanted to run our code without the timer functionality?  We'd have to replace every occurence in our code of `timed_slow_add` with `slow_add`.  If our code spans multiple source files in multiple directories, that can get annoying quickly.  On the other hand, if we instead did `slow_add = timer(slow_add)`, all we'd need to do to remove the functionality is

```python
# slow_add = timer(slow_add)
```

This use pattern is so common that a shorthand has been built into Python's syntax.  Check this out:

In [ ]:
@timer
def slow_multiply(x, y):
    result = x * y
    time.sleep(result)
    return result

slow_multiply(2, 3)

The above code is equivalent to

```python
def slow_multiply(x, y):
    result = x * y
    time.sleep(result)
    return result
slow_multiply = timer(slow_multiply)

slow_multiply(2, 3)
```

Just like above, if we ever wanted to remove the added functionality, all it would take is commenting a line out:

```python
#@timer
def slow_multiply(x, y):
    result = x * y
    time.sleep(result)
    return result
```

Or, better yet,

```python
USE_TIMER = True

def timer(func):
    if not USE_TIMER:
        return func
    
    def new_func(*args, **kwargs):
        start = time.time()
        value = func(*args, **kwargs)
        duration = time.time() - start
        print(f'Running {func} took {duration} seconds.')
        return value
    return new_func
```

This way, all you have to do is change `USE_TIMER` to `False` and *all* functions decorated with `timer` will lose the added functionality.

Combining decorators
--------------------------

You can even apply more than one decorator to a function.  First, let's create a decorator which "logs" (i.e., prints to the screen) the return value of the decorated function.

In [ ]:
def log_result(func):
    def new_func(*args, **kwargs):
        value = func(*args, **kwargs)
        print(f'{func.__name__} returned {value}.')
        return value
    return new_func

@log_result
def returns_five_slowly():
    time.sleep(2)
    return 5

returns_five_slowly()

What if we wanted to print a function's return value *and* how long it took?

In [ ]:
import random

@timer
@log_result
def slow_random_number():
    time.sleep(3)
    return random.randint(1, 10)

slow_random_number()

Pretty neat, huh?  You can chain as many decorators as you want together to add more and more funcionality.

However, look closely at the output.  Does something look strange?

The second line says that `new_func` took three seconds to run.  Why didn't it call the function `slow_random_number` as the first line did?

To see what happened, let's recall what decoration does.  The use of those decorators was equivalent to

```python
def slow_random_number():
    time.sleep(3)
    return random.randint(1, 10)
slow_random_number = log_result(slow_random_number)
slow_random_number = timer(slow_random_number)
```

We're calling `timer` not with the original `slow_random_number` function but with the function that was returned by `log_result`.  If you look at `log_result`'s definition, you see that the returned function was called `new_func`.  That's what was decorated by `log_result`.

That's pretty annoying.  Luckily, there's a way to fix that.  We'll have to redefine our decorators.

In [ ]:
import functools

def timer(func):
    @functools.wraps(func)
    def new_func(*args, **kwargs):
        start = time.time()
        value = func(*args, **kwargs)
        duration = time.time() - start
        print(f'Running {func.__name__} took {duration} seconds.')
        return value
    return new_func

def log_result(func):
    @functools.wraps(func)
    def new_func(*args, **kwargs):
        value = func(*args, **kwargs)
        print(f'{func.__name__} returned {value}.')
        return value
    return new_func

@timer
@log_result
def slow_random_number():
    time.sleep(3)
    return random.randint(1, 10)

slow_random_number()

Much better!  `functools.wraps` modified `new_func` so that it had the same name as `func`.

Decorator factories
-----------------------

Take another look the previous example.  What kind of object is `functools.wraps`?  Is it a decorator?  Not quite.  Take a closer look.  What is decorating `new_func`?  It's not `functools.wraps` but rather `functools.wraps(func)`.  That is, `functools.wraps` took `func` as an argument and *returned* a decorator!  `functools.wraps` is a function which takes a function as an argument and returns a function with takes a function as an argument and returns a function.  Hoo, boy!

`functools.wraps` is what is known as a decorator factory.  Decorator factories are functions which take input and return a decorator to us.  They allow us to create custom decorators upon demand.  They're very useful objects that show up a lot in Python, especially in frameworks such as Flask and pytest.

Let's make one!  Our goal will be to make a decorator factory which accepts a string containing an adjective and returns a decorator which, when applied to a function, will cause that function to describe itself using that adjective whenever called.  That is,

```python
@describe_as('boring')
def does_nothing():
    pass

does_nothing() # prints, "does_nothing is a boring function."
```

You may still be trying to wrap your head around the idea of a function which takes an argument and returns a function which takes a function as an argument and returns a function.  So, before we get to writing the decorator factory, let's write what the decorator would be if we wanted to always use the adjective "boring".

In [ ]:
def describe_as_boring(func):
    @functools.wraps(func)
    def new_func(*args, **kwargs):
        print(f'{func.__name__} is a boring function.')
        return func(*args, **kwargs)
    return new_func

@describe_as_boring
def does_nothing():
    pass

does_nothing()

Pretty easy.  Now, what if we wanted to use a different adjective?

In [ ]:
def describe_as_simple(func):
    @functools.wraps(func)
    def new_func(*args, **kwargs):
        print(f'{func.__name__} is a simple function.')
        return func(*args, **kwargs)
    return new_func

@describe_as_simple
def adds_one(x):
    return x + 1

adds_one(5)

Now, all we need to do is generate the above logic upon demand whenever we're given an adjective.

In [ ]:
def describe_as(adjective):
    article = 'an' if adjective[0] in ('a', 'e', 'i', 'o', 'u') else 'a'
    def decorator(func):
        @functools.wraps(func)
        def new_func(*args, **kwargs):
            print(f'{func.__name__} is {article} {adjective} function.')
            return func(*args, **kwargs)
        return new_func
    return decorator

@describe_as('annoying')
def foobar(value):
    for _ in range(10):
        time.sleep(1)
        print('Blah!')
    return value

foobar(3)

Study the definition of `describe_as` before you move on.  Make sure you understand what each `def` is for.

Appendix
-----------

All of the decorators we've looked at so far have added functionality to the decorated function.  While that is the typical use case, there are times when the decorator returns the original function unmodified.  Why would you ever want to do that?  Sometimes, you want to store a reference to the function for use later (Flask does this with its `app.route` decorator factory).

Consider this simple use case:

In [ ]:
class InvalidCommand(Exception):
    pass

class Robot:
    def __init__(self):
        self.commands = {}
    
    def __getattr__(self, attr):
        command = self.commands.get(attr)
        if command is None:
            raise InvalidCommand(attr)
        return command

    def register_command(self, func):
        self.commands[func.__name__] = func
        return func
    
    def list_commands(self):
        return list(self.commands)

robot = Robot()

@robot.register_command
def sweep_the_floor():
    print('Sweeping the floor')

@robot.register_command
def do_taxes():
    print('Doing your taxes')

@robot.register_command
def tell_a_joke():
    print('Two clowns are eating a cannibal.  One turns to the other and says, "I think we did this joke wrong."')
    
print(robot.list_commands())

If you haven't seen it before, `__getattr__` is a magic method which is called whenever you invoke an attribute of an object which hasn't been explicitly defined.

In [ ]:
robot.sweep_the_floor()

In [ ]:
robot.do_taxes()

In [ ]:
robot.tell_a_joke()